In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from ydata_profiling import ProfileReport

In [2]:
from sentence_transformers import SentenceTransformer
import math
from sklearn.metrics.pairwise import cosine_distances, pairwise_distances, cosine_similarity

In [3]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import cv2
from tqdm.auto import tqdm
import os

In [7]:
import tensorflow as tf
import urllib
import timeit
import requests
from io import BytesIO

In [5]:
from qdrant_client import models, QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.http.models import PointStruct,  CollectionStatus

In [68]:
from transformers import AutoProcessor, CLIPModel
import torch.nn as nn
import pickle

In [7]:
df = pd.read_json('D:\OLD Laptop Data\RADHA-DATA and ML - PROJECT WORK\ML and MLOPS\Hotel Features_Search_SRP\Data\Priyam_FeaturesImages\superappflexidb_dev.hotels_newparetohotels.json')
df.shape

(9131, 5)

In [8]:
df['images'][2]

[{'originalImage': 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/2404/2404970/images/b4e12b2b_z.jpg',
  'wideAngleImage': 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/2404/2404970/images/b4e12b2b_z_w.jpg',
  'thumbNailImage': 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/2404/2404970/images/b4e12b2b_z_tn.jpg',
  'imageAlt': 'Primary image',
  'imageCategoryName': 'Others',
  'imageSubCategoryName': 'Primary image',
  'roomTypeId': None,
  '_id': {'$oid': '653120caf0d1ae2fe023a152'}},
 {'originalImage': 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/2404/2404970/images/cfcaadb4_z.jpg',
  'wideAngleImage': 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/2404/2404970/images/cfcaadb4_z_w.jpg',
  'thumbNailImage': 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/2404/2404970/images/cfcaadb4_z_tn.jpg',
  'imageAlt': 'Interior Entrance',
  'imageCategoryName': '

In [12]:
df.head(2)

,hotelId,amenities,featuredImage,hotelName,images
0,3938571,"[{'amenityId': None, 'amenityNameEn': '24-hour...",[],Havelock Island Beach Resort,[]
1,713805,"[{'amenityId': None, 'amenityNameEn': '24-hour...",[{'originalImage': 'https://fastui.cltpstatic....,TSG Blue Resort & Spa,[{'originalImage': 'https://fastui.cltpstatic....


In [17]:
df['featuredImage'][2]

[{'originalImage': 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/2404/2404970/images/b4e12b2b_z.jpg',
  'wideAngleImage': 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/2404/2404970/images/b4e12b2b_z_w.jpg',
  'thumbNailImage': 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/2404/2404970/images/b4e12b2b_z_tn.jpg',
  'imageAlt': 'Primary image',
  'imageCategoryName': 'Others',
  'imageSubCategoryName': 'Primary image',
  'roomTypeId': None,
  '_id': {'$oid': '653120caf0d1ae2fe023a151'}}]

In [34]:
len(df['featuredImage'][0])

0

In [32]:
df['featuredImage'][2][0]['originalImage']

'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/2404/2404970/images/b4e12b2b_z.jpg'

## Creating a dataframe of Featured Images from main dataframe

In [20]:
df_f = df[['hotelId', 'hotelName']].copy()
df_f.shape

(9131, 2)

In [21]:
df_f.tail(1)

,hotelId,hotelName
9130,4005817,UB Grand Hotel


In [25]:
df_f['originalImage_Featured']=''
df_f['imageAlt_Featured']=''
df_f['imageCategoryName_Featured']=''
df_f['imageSubCategoryName_Featured']=''
df_f['roomTypeId_Featured']=''

In [37]:
df_f['originalImage_Featured']= df['featuredImage'].map(lambda x : x[0]['originalImage'] if len(x)>0 else '')
df_f['imageAlt_Featured']= df['featuredImage'].map(lambda x : x[0]['imageAlt'] if len(x)>0 else '')
df_f['imageCategoryName_Featured']= df['featuredImage'].map(lambda x : x[0]['imageCategoryName'] if len(x)>0 else '')
df_f['imageSubCategoryName_Featured']= df['featuredImage'].map(lambda x : x[0]['imageSubCategoryName'] if len(x)>0 else '')
df_f['roomTypeId_Featured']= df['featuredImage'].map(lambda x : x[0]['roomTypeId'] if len(x)>0 else '')

In [39]:
df_f.head(3)

,hotelId,hotelName,originalImage_Featured,imageAlt_Featured,imageCategoryName_Featured,imageSubCategoryName_Featured,roomTypeId_Featured
0,3938571,Havelock Island Beach Resort,,,,,
1,713805,TSG Blue Resort & Spa,https://fastui.cltpstatic.com/image/upload/hot...,None,Others,Others,None
2,2404970,"Taj Exotica Resort & Spa, Andamans",https://fastui.cltpstatic.com/image/upload/hot...,Primary image,Others,Primary image,None


## Featured Image Classification based on clip model 

- Creating a function which inputs Featured Image URL and outputs its class based on our definition

In [65]:
## Define as global values

import torch
from PIL import Image
from transformers import AutoProcessor, CLIPModel, AutoModel,AutoImageProcessor
import torch.nn as nn
import requests
from io import BytesIO
import urllib.parse

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)

In [45]:
## Define as global 

class_names = ["Bedroom", "Swimming Pool","Dining Room", "Restroom", "Hotel View","Gym","Hotel Facade", "Other", "Reception Area",
              'Bar', 'Garden']

# Compute text features for class names
text_inputs = processor(text=class_names, return_tensors="pt", padding=True).to(device)
with torch.no_grad():
    text_features = model.get_text_features(**text_inputs)

In [46]:
def predict_ThumbnailImageClass(img_url):
    '''
    Image classification function.
    Input : Image url
    Output : Image class
    '''
    imageResponse = requests.get(img_url)  #Image.open(img_url)
    if imageResponse.status_code == 200:  # fetching jpg img
        image1 = Image.open(BytesIO(imageResponse.content))
        with torch.no_grad():
            inputs1 = processor(images=image1, return_tensors="pt").to(device)
            image_features1 = model.get_image_features(**inputs1)
        cos_sim = nn.CosineSimilarity(dim=-1)
        similarity_scores = cos_sim(image_features1, text_features)
    # Get the index of the class with the highest similarity score
        predicted_class_index = torch.argmax(similarity_scores).item()
        return class_names[predicted_class_index] # get the classname  of image

In [47]:
# Adding column for predicted thumbnail image category

df_f['ThumbailImage_PredictedCategory'] = ''

In [86]:
start = timeit.default_timer()

df_f['ThumbailImage_PredictedCategory'] = df_f['originalImage_Featured'].map(lambda x : predict_ThumbnailImageClass(x) if ((x!='') and (x is not None)) else '')

end = timeit.default_timer()
print(end-start)

C:\Users\Radha.Rathore\AppData\Roaming\Python\Python38\site-packages\PIL\Image.py:970: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


12690.434158


In [93]:
## Total time it took to predict class of 9030 images

print(f'{12690/3600} hours to predict class of 9030 images')

3.525 hours to predict class of 9030 images


In [87]:
df_f.tail(3)

,hotelId,hotelName,originalImage_Featured,imageAlt_Featured,imageCategoryName_Featured,imageSubCategoryName_Featured,roomTypeId_Featured,ThumbailImage_PredictedCategory
9128,342389,Sea Executive Suites,https://fastui.cltpstatic.com/image/upload/hot...,Balcony,Others,Primary image,None,Bar
9129,2206632,Abraham Tel Aviv - Hostel,https://fastui.cltpstatic.com/image/upload/hot...,Primary image,Others,Primary image,None,Garden
9130,4005817,UB Grand Hotel,https://fastui.cltpstatic.com/image/upload/hot...,Primary image,Others,Primary image,None,Reception Area


In [88]:
## Saving the predicted Thumbail Class data as pickle
df_f.to_pickle('D:\OLD Laptop Data\RADHA-DATA and ML - PROJECT WORK\ML and MLOPS\Hotel Features_Search_SRP\Data\Priyam_FeaturesImages\OutputFiles\PredictedClass_ThumbnailImage_Data.pkl')

In [90]:
## Saving the predicted Thumbail Class data as excel
df_f.to_excel('D:\OLD Laptop Data\RADHA-DATA and ML - PROJECT WORK\ML and MLOPS\Hotel Features_Search_SRP\Data\Priyam_FeaturesImages\OutputFiles\PredictedClass_ThumbnailImage_Data.xlsx',
             index = False)

In [3]:
## Reading pickle
# df_f2 = pd.read_pickle("D:\OLD Laptop Data\RADHA-DATA and ML - PROJECT WORK\ML and MLOPS\Hotel Features_Search_SRP\Data\Priyam_FeaturesImages\OutputFiles\PredictedClass_ThumbnailImage_Data.pkl")

df_f2 = pd.read_json("D:\OLD Laptop Data\RADHA-DATA and ML - PROJECT WORK\ML and MLOPS\Hotel Features_Search_SRP\Data & Solution\Priyam_FeaturesImages-Data & Solution\OutputFiles\PredictedClass_ThumbnailImage_Data.json")
df_f2.shape

(9131, 8)

In [4]:
df_f2.head(2)

,hotelId,hotelName,originalImage_Featured,imageAlt_Featured,imageCategoryName_Featured,imageSubCategoryName_Featured,roomTypeId_Featured,ThumbailImage_PredictedCategory
0,3938571,Havelock Island Beach Resort,,,,,,
1,713805,TSG Blue Resort & Spa,https://fastui.cltpstatic.com/image/upload/hot...,None,Others,Others,None,Dining Room


## Identify Bad Thumbnail/Featured Images - Restroom, Others, Gym, Reception Area, etc

In [5]:
badThumbnailImages = ["Restroom", "Gym", "Other", "Reception Area",'Bar', '', None]

In [6]:
df_f2['IsThumbnailWrong']= ''

In [7]:
# identifying wrongthumbnail images based on class

df_f2['IsThumbnailWrong']= df_f2['ThumbailImage_PredictedCategory'].map(lambda x : 1 if x in (badThumbnailImages) else 0)

In [8]:
df_f2.tail(1)

,hotelId,hotelName,originalImage_Featured,imageAlt_Featured,imageCategoryName_Featured,imageSubCategoryName_Featured,roomTypeId_Featured,ThumbailImage_PredictedCategory,IsThumbnailWrong
9130,4005817,UB Grand Hotel,https://fastui.cltpstatic.com/image/upload/hot...,Primary image,Others,Primary image,None,Reception Area,1


In [9]:
## Total number of bad thumbnail images
df_f2[df_f2.IsThumbnailWrong == 1].shape

(2429, 9)

In [10]:
## Predicted classes of bad thumbnail images
df_f2[df_f2.IsThumbnailWrong == 1]['ThumbailImage_PredictedCategory'].unique()

array(['', 'Restroom', 'Other', 'Bar', 'Reception Area', 'Gym', None],
      dtype=object)

In [11]:
## Hotel ids list of wrong thumbnail Images

wrongThumbnail_HotelIdList = list(df_f2[df_f2.IsThumbnailWrong == 1]['hotelId'].values)
len(wrongThumbnail_HotelIdList), wrongThumbnail_HotelIdList[:5]

(2429, [3938571, 3961864, 709823, 1361926, 211542])

In [14]:
## Removing extra columsn from output file
df_ClassificationOutput = df_f2.drop(columns=['imageAlt_Featured', 'imageCategoryName_Featured', 'imageSubCategoryName_Featured',
                                             'roomTypeId_Featured'])
df_ClassificationOutput.head(2)

,hotelId,hotelName,originalImage_Featured,ThumbailImage_PredictedCategory,IsThumbnailWrong
0,3938571,Havelock Island Beach Resort,,,1
1,713805,TSG Blue Resort & Spa,https://fastui.cltpstatic.com/image/upload/hot...,Dining Room,0


In [ ]:
## Saving the output forThumbnail Image Classification along with FLAG column (for Tech Team)
df_ClassificationOutput.to_excel('D:\OLD Laptop Data\RADHA-DATA and ML - PROJECT WORK\ML and MLOPS\Hotel Features_Search_SRP\Data & Solution\Priyam_FeaturesImages-Data & Solution\OutputFiles\ClassificationOutput_Flag_Tech.xlsx',
                                index =False)

## Create data for all images of hotels for hotel ids with bad thumbnail images

In [28]:
df_all = df[df['hotelId'].isin(wrongThumbnail_HotelIdList)].copy().reset_index(drop= True)
df_all.shape

(2429, 5)

In [119]:
df_all.tail(2)

,hotelId,amenities,featuredImage,hotelName,images
2427,342389,"[{'amenityId': None, 'amenityNameEn': '24-hour...",[{'originalImage': 'https://fastui.cltpstatic....,Sea Executive Suites,[{'originalImage': 'https://fastui.cltpstatic....
2428,4005817,"[{'amenityId': None, 'amenityNameEn': '24-hour...",[{'originalImage': 'https://fastui.cltpstatic....,UB Grand Hotel,[{'originalImage': 'https://fastui.cltpstatic....


In [120]:
# Creating new dataframe to store all images only for hotel id with bad thumbnails
df_new = df_all[['hotelId','hotelName']]
df_new.tail(1)

,hotelId,hotelName
2428,4005817,UB Grand Hotel


In [57]:
# df_all['images'][5]

In [121]:
## Creating column with images list

df_new['ImagesList'] = ''

for i in range(df_all.shape[0]):
    images = df_all['images'][i]
    imagesList = []
    
    if len(images)>0 :        
        for j in range(len(images)):
#             data = images[j]['originalImage'] + "~" + str(images[j]['imageCategoryName'])+ "~"+ str(images[j]['imageSubCategoryName'])
#             imagesList.append(data) 
            imagesList.append(images[j]['originalImage'])
            
        df_new.at[i, 'ImagesList'] = imagesList
        ''' Pandas AT can insert a list into a cell without raising a ValueError as loc does. 
        I think this is because at always refers to a single value, while loc can refer to values as well as rows and columns.
        '''

<ipython-input-121-f15afcf7b856>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['ImagesList'] = ''


In [122]:
df_new.tail(1)

,hotelId,hotelName,ImagesList
2428,4005817,UB Grand Hotel,[https://fastui.cltpstatic.com/image/upload/ho...


In [128]:
## Saving the lits of images data hotel id wise
df_new.to_pickle("D:\OLD Laptop Data\RADHA-DATA and ML - PROJECT WORK\ML and MLOPS\Hotel Features_Search_SRP\Data\Priyam_FeaturesImages\OutputFiles\ImageList_Hotels.pkl")

In [2]:
## Read df_new
df_new = pd.read_pickle("D:\OLD Laptop Data\RADHA-DATA and ML - PROJECT WORK\ML and MLOPS\Hotel Features_Search_SRP\Data & Solution\Priyam_FeaturesImages-Data & Solution\OutputFiles\ImageList_Hotels.pkl")

In [31]:
print(df_new.shape)
df_new.tail(2)

(2429, 3)


,hotelId,hotelName,ImagesList
2427,342389,Sea Executive Suites,[https://fastui.cltpstatic.com/image/upload/ho...
2428,4005817,UB Grand Hotel,[https://fastui.cltpstatic.com/image/upload/ho...


#### Calling Image Aesthetics Score Utility to fetch top 5 best images for identified hotels 

In [4]:
# Importing ImageAesthetic python utility file

from ImageAestheticSuggestion_Utility import getBestImages

In [26]:
## Creating dataframe chunks
df_new1 = df_new.iloc[:100, :].copy().reset_index(drop = True)
df_new1['Top5Images']= ''
df_new1['BestImage']= ''
print(df_new1.shape)
df_new1.tail(1)

(100, 5)


,hotelId,hotelName,ImagesList,Top5Images,BestImage
99,2052778,Kiranshree Grand,[https://fastui.cltpstatic.com/image/upload/ho...,,


In [15]:
df_new1.loc[2, 'Top5Images'], len(df_new1['ImagesList'][2])

('', 65)

In [17]:
st = timeit.default_timer()

for i in range(df_new1.shape[0]):
    print(i)
    if len(df_new1['ImagesList'][i]) > 0:
        bestImages = getBestImages(df_new1['ImagesList'][i])
        df_new1.at[i, 'Top5Images'] = bestImages
        df_new1.loc[i, 'BestImage'] = bestImages[0]
        df_new1.to_excel('D:\OLD Laptop Data\RADHA-DATA and ML - PROJECT WORK\ML and MLOPS\Hotel Features_Search_SRP\Data & Solution\Priyam_FeaturesImages-Data & Solution\FunctionalityUtilities\ImageAesthetics Utility\ImageAestheticOutputFiles\df_new1.xlsx', index = False)

print(f'{(timeit.default_timer()- st)/3600} hours')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0.0018675999995139136


In [28]:
xxx = pd.read_excel('D:\OLD Laptop Data\RADHA-DATA and ML - PROJECT WORK\ML and MLOPS\Hotel Features_Search_SRP\Data & Solution\Priyam_FeaturesImages-Data & Solution\FunctionalityUtilities\ImageAesthetics Utility\ImageAestheticOutputFiles\df_new1.xlsx')
xxx.shape

(100, 4)

In [30]:
xxx.Top5Iamges[55]

"['https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/2433/2433816/images/image_3724281_original.jpg', 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/2433/2433816/images/image_3724271_original.jpg', 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/2433/2433816/images/image_3724270_original.jpg', 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/2433/2433816/images/image_3724274_original.jpg', 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/2433/2433816/images/image_3724264_original.jpg']"

In [62]:
## Using PANDAS explode to convert convert list of images to rows of images

df_new2 = df_new.explode('ImagesList')
df_new2.reset_index(drop= True, inplace= True)
df_new2.shape

(126793, 3)

In [78]:
# Renaming column
df_new2.rename(columns={'ImagesList': 'OriginalImage'}, inplace= True)

In [173]:
df_new2.tail(3)

,hotelId,hotelName,OriginalImage,ImageAestheticScore
126790,4005817,UB Grand Hotel,https://fastui.cltpstatic.com/image/upload/hot...,
126791,4005817,UB Grand Hotel,https://fastui.cltpstatic.com/image/upload/hot...,
126792,4005817,UB Grand Hotel,https://fastui.cltpstatic.com/image/upload/hot...,


In [80]:
df_new2.hotelId.nunique()

2429

In [87]:
## Saving df_new2 to pickle

df_new2.to_pickle("D:\OLD Laptop Data\RADHA-DATA and ML - PROJECT WORK\ML and MLOPS\Hotel Features_Search_SRP\Data\Priyam_FeaturesImages\OutputFiles\AllImages_BadThumbnail_Hotels.pkl")

In [171]:
## Loading the main hotel file toinclude city column
df_main = pd.read_excel('D:\OLD Laptop Data\RADHA-DATA and ML - PROJECT WORK\ML and MLOPS\Hotel Features_Search_SRP\Hotel_details_WithLatLong.xlsx')
df_main.shape

(194144, 12)

In [172]:
df_main.head(1)

,_id,cityName,stateName,countryName,hotelId,hotelName,locationInfo.latitude,locationInfo.longitude,locationInfo.address,locationInfo.zip,basicInfo.description,ratings.starRating
0,6458a5dc5b7d69a187699ce4,Agra,Uttar Pradesh,India,3924357,Hotel Gopal,27.160808,78.030623,"Near Kailash Cinema, Fatehabad Road, Agra, Utt...",282001,NaN,1.0


In [174]:
dfGoa = df_new2.drop(columns=['ImageAestheticScore'])

In [176]:
dfGoa.shape

(126793, 3)

In [179]:
dfGoa2 = pd.merge(dfGoa, df_main[['hotelId', 'cityName', 'stateName', 'countryName']], left_on = 'hotelId', right_on = 'hotelId',
                 how= 'left')
dfGoa2.shape

(126829, 6)

In [180]:
dfGoa2.head(1)

,hotelId,hotelName,OriginalImage,cityName,stateName,countryName
0,3938571,Havelock Island Beach Resort,,Havelock Island,Andaman and Nicobar Islands,India


In [182]:
dfGoa3 = dfGoa2[dfGoa2.stateName == 'Goa']
dfGoa3.shape

(3888, 6)

In [184]:
dfGoa3.hotelId.nunique()

68

In [183]:
## SAving GOA DATA
dfGoa3.to_excel('D:\OLD Laptop Data\RADHA-DATA and ML - PROJECT WORK\ML and MLOPS\Hotel Features_Search_SRP\Data\GoaHotelImages.xlsx',
               index = False)

## Logic to find better thumbnail Image - Image Aesthetic Score

- Function with input - hotel id and list of all images in that hotel
- Call Suresh's function to give aesthetic score for images of that hotel id, for selected categories (facade, bedroom, etc)
- Output - give image to be shown as thumbnail

In [88]:
df_new2 = pd.read_pickle("D:\OLD Laptop Data\RADHA-DATA and ML - PROJECT WORK\ML and MLOPS\Hotel Features_Search_SRP\Data\Priyam_FeaturesImages\OutputFiles\AllImages_BadThumbnail_Hotels.pkl")

In [66]:
def cosine_similarity(vec1, vec2):
    # Compute the dot product of vec1 and vec2
    dot_product = np.dot(vec1, vec2)
    
    # Compute the L2 norm of vec1 and vec2
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    
    # Compute the cosine similarity
    similarity = dot_product / (norm_vec1 * norm_vec2)
    
    return similarity

In [69]:
with open('D:\OLD Laptop Data\RADHA-DATA and ML - PROJECT WORK\ML and MLOPS\Hotel Features_Search_SRP\Data\Priyam_FeaturesImages\ImageAestheticsInputFiles\hotel_positive_prompt.pkl', 'rb') as f:
    average_positive_vector = pickle.load(f)
with open('D:\OLD Laptop Data\RADHA-DATA and ML - PROJECT WORK\ML and MLOPS\Hotel Features_Search_SRP\Data\Priyam_FeaturesImages\ImageAestheticsInputFiles\hotel_negative_prompt.pkl', 'rb') as f:
    average_negative_vector = pickle.load(f)

In [74]:
def predict_ImageAestheticScore(img_url):
    imageResponse = requests.get(img_url)  #Image.open(img_url)
    if imageResponse.status_code == 200:  # fetching jpg img
        image1 = Image.open(BytesIO(imageResponse.content))
    
#     image1 = load_image_PIL(img_url)
    with torch.no_grad():
        inputs1 = processor(images=image1, return_tensors="pt").to(device)
        image_features1 = model.get_image_features(**inputs1)
    image_vector=image_features1.numpy()
    positive_similarity=cosine_similarity(average_positive_vector,np.transpose(image_vector))
    negative_similarity=cosine_similarity(average_negative_vector,np.transpose(image_vector))
    aesthetic_score=positive_similarity+(-1*negative_similarity)
    return aesthetic_score*1000

In [81]:
# start = timeit.default_timer()
# predict_ImageAestheticScore(df_new2['OriginalImage'][1])
# end = timeit.default_timer()
# print(end-start)

1.3604267999999138


In [89]:
(1.50*126793)/3600  ## Time for all 1.26 lakhs images = 52 hours

52.830416666666665

In [82]:
## creating column for Aesthetic Score of image
df_new2['ImageAestheticScore'] = ''

In [91]:
# start = timeit.default_timer()

# df_new2['ImageAestheticScore'] = df_new2['OriginalImage'].map(lambda x : predict_ImageAestheticScore(x) if ((x!='') and (x is not None)) else '')

# end = timeit.default_timer()
# print(end-start)

# 1. Utility - Image Thumbnail

In [102]:
df_new2['OriginalImage'][1]

'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/3961/3961864/images/image_3961864_2ef88de5-f9be-4211-8bea-599afbed3d69.jpeg'

In [109]:
# Format !python filename url

!python "D:\OLD Laptop Data\RADHA-DATA and ML - PROJECT WORK\ML and MLOPS\Hotel Features_Search_SRP\Data\Priyam_FeaturesImages\FunctionalityUtilities\ImageDuplicationIdentification.py" "https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/3961/3961864/images/image_3961864_2ef88de5-f9be-4211-8bea-599afbed3d69.jpeg"

Hi
['Restroom', 'Gym', 'Other', 'Reception Area', 'Bar', '', None]
https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/3961/3961864/images/image_3961864_2ef88de5-f9be-4211-8bea-599afbed3d69.jpeg
2
Restroom


In [111]:
# a,b  =os.system("D:\OLD Laptop Data\RADHA-DATA and ML - PROJECT WORK\ML and MLOPS\Hotel Features_Search_SRP\Data\Priyam_FeaturesImages\FunctionalityUtilities\ImageDuplicationIdentification.py https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/3961/3961864/images/image_3961864_2ef88de5-f9be-4211-8bea-599afbed3d69.jpeg")

In [165]:
import subprocess

# Replace 'your_script.py' with the path to your Python script.
python_script = 'D:\OLD Laptop Data\RADHA-DATA and ML - PROJECT WORK\ML and MLOPS\Hotel Features_Search_SRP\Data\Priyam_FeaturesImages\FunctionalityUtilities\ImageDuplicationIdentification.py'

# Replace 'param1' and 'param2' with your input parameters.
param1 = df_new2['OriginalImage'][1]

# Construct the command to run the Python script with input parameters.
cmd = ['python', python_script, param1]

# Run the command and capture the output in a variable.
output = subprocess.check_output(cmd, universal_newlines=True)
print(str(output))

Restroom-1



In [167]:
# Replace 'your_script.py' with the path to your Python script.
python_script = 'D:\OLD Laptop Data\RADHA-DATA and ML - PROJECT WORK\ML and MLOPS\Hotel Features_Search_SRP\Data\Priyam_FeaturesImages\FunctionalityUtilities\ImageDuplicationIdentification.py'

# Replace 'param1' and 'param2' with your input parameters.
param1 = df_new2['OriginalImage'][1]

# Construct the command to run the Python script with input parameters.
cmd = ['python', python_script, param1]
# Call the subprocess
result = subprocess.run(cmd, stdout=subprocess.PIPE)
print(result)

# Get the output and decode it
output = result.stdout.decode('utf-8')
output

CompletedProcess(args=['python', 'D:\\OLD Laptop Data\\RADHA-DATA and ML - PROJECT WORK\\ML and MLOPS\\Hotel Features_Search_SRP\\Data\\Priyam_FeaturesImages\\FunctionalityUtilities\\ImageDuplicationIdentification.py', 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/3961/3961864/images/image_3961864_2ef88de5-f9be-4211-8bea-599afbed3d69.jpeg'], returncode=0, stdout=b'Restroom\r\n')


'Restroom\r\n'

# UTILITY - IMAGE AESTHETIC SCORE

In [170]:
# str(df_new['ImagesList'][200])[1:-1].split(',')

In [143]:
str(df_new['ImagesList'][200])[1:-1].split(',')[3].strip()[1:-1].strip()
# imgList = [i.strip()[1:-1].strip() for i in df3['UniqueImagesList'][j][1:-1].split(',')]

'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/3974/3974419/images/image_2805615_original.jpg'

In [150]:
## TEST 1
# Replace 'your_script.py' with the path to your Python script.
python_script = 'D:\OLD Laptop Data\RADHA-DATA and ML - PROJECT WORK\ML and MLOPS\Hotel Features_Search_SRP\Data\Priyam_FeaturesImages\FunctionalityUtilities\ImageAestheticScore.py'

# Input parameter is list of images asa string. 
param1 = ''

# Construct the command to run the Python script with input parameters.
cmd = ['python', python_script, param1]

# Run the command and capture the output in a variable.
output = subprocess.check_output(cmd, universal_newlines=True)
print(output)

No images found.
{}



In [151]:
## TEST 2

# Input parameter is list of images asa string. 
param1 = str(df_new['ImagesList'][200])

# Construct the command to run the Python script with input parameters.
cmd = ['python', python_script, param1]

# Run the command and capture the output in a variable.
output = subprocess.check_output(cmd, universal_newlines=True)
print(output)

{'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/3974/3974419/images/image_2805613_original.jpg': 32.394497, 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/3974/3974419/images/image_2805646_original.jpg': 32.394497, 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/3974/3974419/images/image_2805614_original.jpg': 32.116398, 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/3974/3974419/images/image_2805645_original.jpg': 32.116398, 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/3974/3974419/images/image_2805616_original.jpg': 12.888193, 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/3974/3974419/images/image_2805615_original.jpg': 12.835041, 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/3974/3974419/images/image_2805617_original.jpg': 7.888332, 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/3974/3974419/images/image_28

In [169]:
# ## TEST 2

# # Input parameter is list of images asa string. 
# param1 = str(df_new['ImagesList'][200])

# # Construct the command to run the Python script with input parameters.
# cmd = ['python', python_script, param1]

# # Run the command and capture the output in a variable.
# # output = subprocess.check_output(cmd, universal_newlines=True)
# # print(output)
# try:
#     subprocess.check_output(cmd , shell=True, stderr=subprocess.STDOUT)
# except subprocess.CalledProcessError as e:
#     raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))
    
    # Input parameter is list of images asa string. 
param1 = str(df_new['ImagesList'][200])

# Construct the command to run the Python script with input parameters.
cmd = ['python', python_script, param1]

# Run the command and capture the output in a variable.
output = subprocess.check_output(cmd, universal_newlines=True)
print(output)

CalledProcessError: Command '['python', 'D:\\OLD Laptop Data\\RADHA-DATA and ML - PROJECT WORK\\ML and MLOPS\\Hotel Features_Search_SRP\\Data\\Priyam_FeaturesImages\\FunctionalityUtilities\\ImageDuplicationIdentification.py', "['https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/3974/3974419/images/image_2805612_original.jpg', 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/3974/3974419/images/image_2805613_original.jpg', 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/3974/3974419/images/image_2805614_original.jpg', 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/3974/3974419/images/image_2805615_original.jpg', 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/3974/3974419/images/image_2805616_original.jpg', 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels/cms/3974/3974419/images/image_2805617_original.jpg', 'https://fastui.cltpstatic.com/image/upload/hotels/places/hotels